In [ ]:
import pyspark
from operator import add

Intialize a spark instance-

In [ ]:
sc = pyspark.SparkContext(appName="spark-notebook")

In [ ]:
myRDD = sc.textFile("file:///path/to/spark-notebook/num.txt", 10)

Get number of RDD partitions-

In [ ]:
myRDD.getNumPartitions()

In [ ]:
print myRDD.take(20) # get first 20 values

We will define a square function for our map operation- 

In [ ]:
def square(value):
    return int(value)**2

In [ ]:
newRDD = myRDD.map(square)

In [ ]:
print newRDD.collect() # get map results

In [ ]:
subRDD = newRDD.map(lambda x: (x, 1) if x%2==0 else (x, 0)) # using lamda functions

The above map function maps generated two types of key value pairs- (even, 1) and (odd, 0)

In [ ]:
print subRDD.collect()

Now define a lambda function to generate (key, value) pairs where key = even or odd depending upon the input and 
value = input

In [ ]:
testRDD = # your code here

In [ ]:
print testRDD.take(10)

There are two types of Reduce operations- reduceByKey() and reduce(). Check PySpark documentation for more details

In [ ]:
reduced = testRDD.reduceByKey(add)

In [ ]:
print reduced.collect()